In [47]:
from utils.ClientBase import ClientBase
import pandas as pd
import json

In [3]:
client = ClientBase()

In [6]:
folder = '~/Desktop/projects/nft-music/src/'

## Genre Based

In [52]:
string = """
    select
      artist_id as name,
      date,
      (((streams - minval)::float / (maxval - minval)::float) * 100)::int as streams_norm
    from (
      select
        s.artist_id,
        s.streams,
        s.date,
        max(streams) over (partition by s.artist_id) as maxval,
        min(streams) over (partition by s.artist_id) as minval
      from (
        select a.artist_id
        from nielsen_genres.artists a
        left join nielsen_artist.stats st on a.artist_id = st.artist_id
        where genre = 'r&b'
        order by tw_streams desc nulls last
        limit 100
      ) q
      left join nielsen_artist.streams s on q.artist_id = s.artist_id
      where date >= '2021-01-01'
          and date <= '2021-12-31'
    ) q
"""
df = client.execute(string)

In [53]:
data = df.groupby('date').apply(lambda x: x[['name', 'streams_norm']].to_dict('records')).values.tolist()

In [54]:
data

[[{'name': 52, 'streams_norm': 46},
  {'name': 54, 'streams_norm': 33},
  {'name': 75, 'streams_norm': 30},
  {'name': 78, 'streams_norm': 26},
  {'name': 81, 'streams_norm': 0},
  {'name': 102, 'streams_norm': 8},
  {'name': 105, 'streams_norm': 38},
  {'name': 109, 'streams_norm': 2},
  {'name': 131, 'streams_norm': 12},
  {'name': 149, 'streams_norm': 8},
  {'name': 158, 'streams_norm': 1},
  {'name': 169, 'streams_norm': 27},
  {'name': 180, 'streams_norm': 16},
  {'name': 185, 'streams_norm': 40},
  {'name': 203, 'streams_norm': 21},
  {'name': 207, 'streams_norm': 56},
  {'name': 217, 'streams_norm': 2},
  {'name': 220, 'streams_norm': 15},
  {'name': 264, 'streams_norm': 65},
  {'name': 265, 'streams_norm': 9},
  {'name': 268, 'streams_norm': 9},
  {'name': 270, 'streams_norm': 22},
  {'name': 278, 'streams_norm': 76},
  {'name': 279, 'streams_norm': 5},
  {'name': 325, 'streams_norm': 0},
  {'name': 326, 'streams_norm': 19},
  {'name': 339, 'streams_norm': 2},
  {'name': 378, '

In [59]:
with open(filename, 'w') as outfile:
    json.dump(data, outfile)